In [1]:
import time
import os
import torch
from sbi import utils as utils
from sbi.inference import SNPE, SNPE_A, SNPE_C
from sbi.utils.user_input_checks import process_prior

from simulations_model1 import simulator_sbi_entropy
from parsing_functions import save_pickle_data, load_input_data, extract_specs

## Loading dataset

In [2]:
save_dir = 'saved_datasets/'
dataset_name = 'dataset_5000sim_5e+04np_1e-02dt_10os_1e+02pre.pickle'

dataset_path = os.path.join(save_dir, dataset_name)

x, theta = load_input_data(input_file = dataset_path)

# Extract the specifications
specs_dict = extract_specs(dataset_name)

# Unpack the specifications from the dictionary
num_simulations = specs_dict['num_simulations']
Npts = specs_dict['Npts']
dt = specs_dict['dt']
oversampling = specs_dict['oversampling']
prerun = specs_dict['prerun']
prefix = specs_dict['prefix']

# Saving string for future saving
specs = f'{prefix}_{num_simulations:.0f}sim_{Npts:.0e}np_{dt:.0e}dt_{oversampling}os_{prerun:.0e}pre'
print(specs)

None_5000sim_5e+04np_1e-02dt_10os_1e+02pre


In [3]:
low2_tensor = torch.tensor([0,0.1,0])
high2_tensor = torch.tensor([4,4,4])
prior_sbi = utils.BoxUniform(low=low2_tensor, high=high2_tensor)

# Check prior, return PyTorch prior.
prior, num_parameters, prior_returns_numpy = process_prior(prior_sbi)

## Inference

In [4]:
inference = SNPE(prior=prior)
inference = inference.append_simulations(theta, x)
start = time.time()
density_estimator = inference.train()
posterior = inference.build_posterior(density_estimator)
end = time.time()
print('\n time passed for convergence: ', end-start)

 Neural network successfully converged after 133 epochs.
 time passed for convergence:  95.65190601348877


In [5]:
#saving generated posterior from SBI
posterior_name = 'posterior_' + str(specs)
save_dir = 'saved_posteriors'

save_pickle_data(data=posterior, filename=posterior_name, folder_path=save_dir)

Saved dataset at saved_posteriors/posterior_None_5000sim_5e+04np_1e-02dt_10os_1e+02pre.pickle

